# 1

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import json
import numpy as np

!wget --no-check-certificate \
    https://storage.googleapis.com/learning-datasets/sarcasm.json \
    -O /tmp/sarcasm.json


def pad (data):
  return pad_sequences(data, maxlen=1000, truncating="post", padding="post")


training_size = 20000

sentences = []
labels = []

with open ("/tmp/sarcasm.json", "r") as f:
  datastore = json.load(f)


for i in datastore:
  sentences.append(i["headline"])
  labels.append(i["is_sarcastic"])

training_sents = sentences[0:training_size]
test_sents = sentences[training_size:]

training_labels = labels [0:training_size]
test_labels = labels [training_size:]

tokenizer = Tokenizer (
  oov_token = "<OOV>",
  num_words = 10000
)
tokenizer.fit_on_texts(training_sents)

wordindex = tokenizer.word_index

training_seq = tokenizer.texts_to_sequences(training_sents)
pad_training_seq = pad(training_seq)

testing_seq = tokenizer.texts_to_sequences(test_sents)
pad_testing_seq = pad(testing_seq)

testing_padded = np.array(pad_testing_seq)
training_padded = np.array(pad_training_seq)
training_labels = np.array(training_labels)
testing_labels = np.array(test_labels)

model = tf.keras.Sequential([
  tf.keras.layers.Embedding(10000, 16, input_length = 1000),
  tf.keras.layers.GlobalAveragePooling1D(),
  tf.keras.layers.Dense(24, activation="relu"),
  tf.keras.layers.Dense(1, activation="sigmoid")
])

model.compile(
  loss = "binary_crossentropy",
  optimizer = "adam",
  metrics = ["accuracy"]
)

# 2

In [ ]:
history = model.fit(
  training_padded,
  training_labels,
  epochs=30,
  validation_data = (testing_padded, testing_labels),
  verbose = 2
)

# 3

In [ ]:
while True:
  sentence = input ("You: ")

  sequences = pad(tokenizer.texts_to_sequences([sentence]))

  print(f"{str(model.predict(sequences)[0][0])[2:4]}% sarcastic")